El objetivo de esta sección es mostrar cómo se pueden desarrollar flujos de preprocesamiento en `sklearn`. Vamos a explorar las diferentes operaciones que se le pueden hacer a los diferentes tipos de variables y cómo agrupar todas las operaciones en un solo elemento de `sklearn` que tenga los métodos `.fit()` y `.transform()`.

In [36]:
import numpy as np
import pandas as pd

from typing import List
from sklearn import set_config
set_config(display='diagram')

In [37]:
data = pd.read_csv(
    "https://factored-workshops.s3.amazonaws.com/taxi-trip-duration.csv"
)
# Limitar rango de datos
tiempo_minimo = 60 # 1 minuto
tiempo_maximo = 36000 # 10 horas
data = data[
    (data["trip_duration"] > tiempo_minimo) &
    (data["trip_duration"] < tiempo_maximo)
]

data.shape


(1447855, 13)

In [38]:
data.head()

,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration,pickup_borough,dropoff_borough
0,id2875421,2,2016-03-14 17:24:55,2016-03-14 17:32:30,1,-73.982155,40.767937,-73.964630,40.765602,N,455,Manhattan,Manhattan
1,id2377394,1,2016-06-12 00:43:35,2016-06-12 00:54:38,1,-73.980415,40.738564,-73.999481,40.731152,N,663,Manhattan,Brooklyn
2,id3858529,2,2016-01-19 11:35:24,2016-01-19 12:10:48,1,-73.979027,40.763939,-74.005333,40.710087,N,2124,Manhattan,Brooklyn
3,id3504673,2,2016-04-06 19:32:31,2016-04-06 19:39:40,1,-74.010040,40.719971,-74.012268,40.706718,N,429,Brooklyn,Brooklyn
4,id2181028,2,2016-03-26 13:30:55,2016-03-26 13:38:10,1,-73.973053,40.793209,-73.972923,40.782520,N,435,Manhattan,Manhattan


Es importante siempre separar la variable dependiente—en este caso `trip_duration` del dataframe que vamos a usar para crear las variables independientes.

In [39]:
#Queremos saber el tiempo de la persona en su viaje. queremos predecir
# el tiempo

y = data["trip_duration"]
x = data.drop(
    ["id", "trip_duration", "dropoff_datetime", "store_and_fwd_flag"],
    axis="columns"
)


#Drop == Eliminamos columnas:
# "id": identificador unico, para el modelo no es relevante, no brinda infromación
# "trip_duration": variable a precedir entonces no se debe repetir.
# "dropoff_datetime": tiempo que acabo el viaje, no tendremos que dia acabo ni a que hora acabo, queremos predecir ese tiempo.
# "store_and_fwd_flag": varible bandera, se si almacenar o no el viaje

store_and_fwd_flag
Esta bandera indica si el registro de viaje se mantuvo en la memoria del vehículo antes de enviarlo al proveedor porque el vehículo no tenía conexión con el servidor - Y= almacer y enviar; N= no es un viaje de almacenamiento y reenvío

In [40]:
data['store_and_fwd_flag'].value_counts()

N    1439947
Y       7908
Name: store_and_fwd_flag, dtype: int64

División de los datos

In [41]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x,
                                                    y,
                                                    #test_size=0.25
                                                    random_state=0)

# test_size = 0.25 -- default

Transformers

scikit-learn incluye una gran lista de transformers que nos permiten limpiar y transformar datos dependiendo de nuestro objetivo. Sin importar el tipo de transformacion que apliquen, todos los transformers siguen la convención de tener por lo menos dos métodos:
    .fit() calcula los parámetros necesarios para realizar la transformación a partir de unos datos de entrada. Este método se ejecuta únicamente en los datos de entrenamiento para asegurarnos que los parámetros no contienen información de los datos de validación.
    .transform() aplica la transformación a los datos.
Veamos un ejemplo usando StandardScaler, un transformer que nos permite remover la media y escalar los datos para que tengan varianza de 1. Vamos a usarlo para normalizar los coordenadas de inicio del viaje.


Transformación, para limpiar datos. 
El fit es para entrenar, aprende los parámetros
Para utilizarlos en el transformer
StandardScaler: para normalizar nuestra data.

In [42]:
from sklearn.preprocessing import StandardScaler

transformer = StandardScaler()
transformer.fit(
    x_train[["pickup_longitude", "pickup_latitude"]]
)
normed_array = transformer.transform(
    x_test[["pickup_longitude", "pickup_latitude"]]
)
print(normed_array)

[[-0.30923835 -0.27478234]
 [-0.13589444  0.75675283]
 [ 0.24599071  0.58169127]
 ...
 [-0.19727957 -0.13002453]
 [-0.03955223 -0.08161096]
 [-0.11150857 -0.79791458]]


Custom Transformers

A pesar de que scikit-learn ofrece varias operaciones para transformar datos, frecuentemente necesitamos crear transformaciones que son específicas a nuestro proyecto. Para esto vamos a aprender cómo implementar custom transformers.

Todos los transformer custom deben heredar BaseEstimator y TransformerMixin para que tengan todas las funciones necesarias para conectarse a otros objetos de sklearn. Por convención de sklearn, los objetos usados para transformar datos siempre deben tener los métodos .fit() y .transform(). Ambos métodos reciben X y y para que se integren sin problemas a pipelines de sklearn. 

El método .fit() sirve para almacenar cantidades que vamos a usar durante la transformación de los datos y siempre debe retornar self. El método .transform() ejecuta la transformación y retorna los datos transformados. Ahora vamos a replicar el StandardScaler pero esta vez escribiéndolo como un transformer personalizado que no retorne un array sino un DataFrame.

Saca la media y le divide la desviación estándar
Se pierde o no se pierde información.
Achica el min y el max. Lo que hace es que uno maneja unos rangos más amplios que otros 


In [43]:
from sklearn.base import BaseEstimator, TransformerMixin

class PrimerTransformer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        self.mean = X.mean()
        self.std = X.std()
        return self

    def transform(self, X, y=None):
        return (X - self.mean) / self.std

Checkpoint # 1

Correr .fit() y .transform() para PrimerTransformer para las coordenadas de inicio del viaje y verificar que el resultado sea igual al del StandardScaler. 

De PrimerTransformer va a salir un DataFrame en lugar de un array pero los valores deben ser los mismos.



In [44]:
'''primer_transformer = PrimerTransformer()
train_normed_df = primer_transformer.fit_transform(
train_df[["pickup_longitude", "pickup_latitude"]]
)
val_normed_df = primer_transformer.transform(
val_df[["pickup_longitude", "pickup_latitude"]]
)
'''

'primer_transformer = PrimerTransformer()\ntrain_normed_df = primer_transformer.fit_transform(\ntrain_df[["pickup_longitude", "pickup_latitude"]]\n)\nval_normed_df = primer_transformer.transform(\nval_df[["pickup_longitude", "pickup_latitude"]]\n)\n'

En <a id='section_id'> este </a> link pueden encontrar más detalles de qué papel juegan los métodos __init__, .fit() y .transform()[esto es un link](www.google.com)

In [45]:
pt = PrimerTransformer()

pt.fit(x_train[["pickup_longitude", "pickup_latitude"]])
pt.transform(x_test[["pickup_longitude", "pickup_latitude"]])

,pickup_longitude,pickup_latitude
5949,-0.309238,-0.274782
255492,-0.135894,0.756752
979171,0.245991,0.581691
121356,0.176797,0.385864
524382,0.194096,0.182913
...,...,...
885062,-0.488949,-0.292409
755237,-0.565470,-1.408094
790367,-0.197279,-0.130024
106875,-0.039552,-0.081611


Transformer Fechas

Ahora que sabemos cómo construir objetos para transformar datos, vamos a crear un transformer para crear variables como el día de la semana y la hora del momento en el que empieza el servicio. Como vimos la semana pasada en nuestro EDA, esa información puede ser relevante para determinar la duración del viaje.

En nuestra transformación no debemos almacenar ningún dato para hacer las transformación entonces dejamos el método .fit() vacío. En .transform() extraemos los datos de fecha que nos interesan y retornamos un dataframe con las nuevas variables.

In [46]:
class TransformerFechas(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        columna_fecha = pd.to_datetime(X["pickup_datetime"])
        fecha_df = pd.DataFrame()
        # TODO: Crear columnas con dia de la semana y hora de recogida.
        fecha_df["weekday"] = columna_fecha.dt.weekday
        fecha_df["hour"] = columna_fecha.dt.hour
        return fecha_df

In [47]:
x_train.columns

Index(['vendor_id', 'pickup_datetime', 'passenger_count', 'pickup_longitude',
       'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude',
       'pickup_borough', 'dropoff_borough'],
      dtype='object')

Checkpoint # 2.1

Completar el codigo para extrar el día de la semana y la hora a partir de una fecha en pandas y los ponga en las columnas weekday y hour.

In [48]:
transformer_fechas = TransformerFechas()
fechas_df = transformer_fechas.fit_transform(x_train)
fechas_df.head()

,weekday,hour
518949,3,21
1128931,6,21
574396,1,18
54790,6,17
599130,0,16


In [49]:
x_train["pickup_datetime"].head(3)

518949     2016-06-02 21:41:39
1128931    2016-03-27 21:59:43
574396     2016-02-02 18:13:10
Name: pickup_datetime, dtype: object

Transformer Distancia

También queremos crear una feature que nos ayude a medir la distancia entre el punto de origen y el punto de destino usando la longitud y la latitud en los datos. Nuevamente no tenemos que almacenar cantidades en nuestro método .fit() y calculamos la distancia entre los dos puntos usando la distancia de Haversine. En el código ya está implementada la función para calcular la distancia y no es necesario fijarse en los detalles de la implementación. Como pueden ver, nuestros transformers pueden incluir funciones adicionales que nos ayuden a calcular cantidades útiles para realizar la transformación.

In [50]:
class TransformerDistancia(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        X_init = X[["pickup_latitude", "pickup_longitude"]].to_numpy()
        X_final = X[["dropoff_latitude", "dropoff_longitude"]].to_numpy()

        # Distancia de Haversine
        # TODO: Calcular la variable distancia usando la funcion
        # distancia de Haversine.
        distancia = self.distancia_haversine(X_init = X_init, X_final = X_final)
        distancia_df = pd.DataFrame()
        distancia_df["distancia"] = distancia
        return distancia_df
    
    # DISTANCIA GEOESPACIALES - LATITUD/LONGITUD
    def distancia_haversine(self, X_init, X_final):
        # Convertir de decimal a radianes
        X_init = np.radians(X_init)
        X_final = np.radians(X_final)

        # Formula Haversine
        dlat = X_final[:, 0] - X_init[:, 0] 
        dlon = X_final[:, 1] - X_init[:, 1]
        a = np.sin(dlat / 2) ** 2 + np.cos(X_init[:, 0]) * (
            np.cos(X_final[:, 0])) * (
            np.sin(dlon / 2) ** 2)
        c = 2 * np.arcsin(np.sqrt(a))
        r = 6371
        # Radius of earth in kilometers. Use 3956 for miles. Determines return value units.
        # Determines return value units.
        return c * r

Checkpoint # 2.2

Calcular la distancia de Haversine usando el método que está implementado en la clase y almacenarlo en la variable distancia.

In [51]:
transformer_dist = TransformerDistancia()
distancias_df = transformer_dist.fit_transform(x_train)
distancias_df.head()

,distancia
0,2.404355
1,0.390267
2,5.629826
3,4.298386
4,7.488963


Unión de transformers con Pipelines

Pipeline Numérico

    Ahora vamos a usar los objetos de Pipeline y ColumnTransformer para unir los transformers que ya hemos creado con otros disponibles en sklearn y lograr todas las transformaciones que queremos realizar a todos los datos.

In [52]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

    ColumnTransformer nos permite elegir las columnas sobre las que queremos aplicar una transformación cuando nos llegan columnas adicionales. En este caso queremos que al TransformerDistancia llegue únicamente pickup_longitude, pickup_latitude, dropoff_longitude y dropoff_latitude.

    Leyendo la documentación sabemos que debemos pasar una tupla con el nombre del transformer, la clase que define el transformer y las columnas sobre las que queremos aplicar la transformación. ColumnTransformer también nos permite definir qué se debe hacer con las columnas que no estamos transformando; en este caso elegimos pasarlas sin transformarlas remainder="passthrough".

In [53]:
coord_cols = [
    "pickup_longitude",
    "pickup_latitude",
    "dropoff_longitude",
    "dropoff_latitude"
]

transformer_coord = ColumnTransformer(
    [
        ("transformer_dist", TransformerDistancia(), coord_cols),
    ],
    remainder="passthrough" # las demas variable no sean transformadas
)
display(transformer_coord)

ColumnTransformer(remainder='passthrough',
                  transformers=[('transformer_dist', TransformerDistancia(),
                                 ['pickup_longitude', 'pickup_latitude',
                                  'dropoff_longitude', 'dropoff_latitude'])])

Ahora que tenemos nuestro transformer que realiza transformación usando las coordenadas y deja pasar columnas adicionales, vamos a usarlo para unirlo con la otra variable númerica que tenemos en el dataset—passenger_count— y normalizarlas usando StandardScaler. Para esto vamos a usar el objeto Pipeline.

Pipeline (documentación Pipeline) nos permite concatenar transformaciones de sklearn. En este caso vamos a concatenar el ColumnTransformer que creamos con TransformerDistancia con el StandardScaler. Como passenger_count no estaba entre columnas que selecciona transformer_coord, esa columna pasa directamente a ser normalizada por el StandardScaler.

In [54]:
num_cols = ["passenger_count"] + coord_cols

num_pipeline = Pipeline(
    [
        ("transformer_coord", transformer_coord), # distancia / los 2 puntos
        ("scaler", StandardScaler())
    ]
)

X_num = num_pipeline.fit_transform(x_train[num_cols], y_train)
print(X_num)

# Tenemos cantidad de pasajeros
# y distancia

2021/10/23 00:19:24 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID 'e2b948ab438f4ce9a75604c75245a79c', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current sklearn workflow
2021/10/23 00:19:24 WARNING mlflow.utils: Truncated the value of the key `steps`. Truncated value: `[('transformer_coord', ColumnTransformer(remainder='passthrough',
                  transformers=[('transformer_dist', TransformerDistancia(),
                                 ['pickup_longitude', 'pickup_latitude',
                               ...`
2021/10/23 00:19:24 WARNING mlflow.utils: Truncated the value of the key `transformer_coord`. Truncated value: `ColumnTransformer(remainder='passthrough',
                  transformers=[('transformer_dist', TransformerDistancia(),
                                 ['pickup_longitude', 'pickup_latitude',
                                  'dropoff_longitude',...`
2021/10/23 00:19:25 WAR

[[-0.25154928 -0.50616698]
 [-0.73141214  0.25385023]
 [ 0.51692963 -0.50616698]
 ...
 [-0.41058443 -0.50616698]
 [-0.3461049  -0.50616698]
 [ 1.33898968  0.25385023]]


In [55]:
display(num_pipeline)

Pipeline(steps=[('transformer_coord',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('transformer_dist',
                                                  TransformerDistancia(),
                                                  ['pickup_longitude',
                                                   'pickup_latitude',
                                                   'dropoff_longitude',
                                                   'dropoff_latitude'])])),
                ('scaler', StandardScaler())])

# Pipeline Categórico

Usaremos una lógica similar para las variables categóricas pero esta vez para concatenar el resultado de extraer variables temporales de la fecha de inicio del viaje con un OneHotEncoder para las variables categóricas de los datos.

In [56]:
from sklearn.preprocessing import OrdinalEncoder

cat_cols = ["vendor_id", "pickup_borough", "pickup_datetime"]

transformer_fechas = ColumnTransformer(
    [
        #TODO: Punto 1 de Checkpoint 3
        ("transformer_fechas", TransformerFechas(),['pickup_datetime'])
    ],
    remainder="passthrough"
)

cat_pipeline = Pipeline(
    [
        #TODO: Punto 2 de Checkpoint 3
        ("transformer_fechas", transformer_fechas),
        ("OrdinalEncoder", OrdinalEncoder()) # convierte de letras a numeros. 
    ]
)

X_cat = cat_pipeline.fit_transform(x_train[cat_cols])
print(X_cat)

2021/10/23 00:19:31 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID 'eea09046af954aeb9352be50071803f7', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current sklearn workflow
2021/10/23 00:19:36 WARNING mlflow.sklearn: Training metrics will not be recorded because training labels were not specified. To automatically record training metrics, provide training labels as inputs to the model training function.
2021/10/23 00:19:36 WARNING mlflow.sklearn: Failed to infer model signature: the trained model does not specify a `predict` function, which is required in order to infer the signature


[[ 3. 21.  0.  1.]
 [ 6. 21.  1.  2.]
 [ 1. 18.  1.  2.]
 ...
 [ 6.  2.  0.  3.]
 [ 1. 23.  1.  1.]
 [ 1.  2.  1.  2.]]


## Checkpoint # 3

Crear un ColumnTransformer llamado transformer_fechas que solo seleccione la variable pickup_datetime para aplicarle el TransformerFechas.
Unir el ColumnTransformer del punto 1 con un OrdinalEncoder para transformar las variables que salen del ColumnTransformer de fechas. 

In [57]:
display(cat_pipeline)

Pipeline(steps=[('transformer_fechas',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('transformer_fechas',
                                                  TransformerFechas(),
                                                  ['pickup_datetime'])])),
                ('OrdinalEncoder', OrdinalEncoder())])

Unión de Pipelines

    Por último, usaremos nuevamente el ColumnTransformer para determinar cuáles son las variables numéricas y las variables catégoricas en nuestros datos. Definiendo varios elementos en la lista que le pasamos al ColumnTransformer le hacemos saber a sklearn que queremos diferentes transformaciones para las columnas y al final queremos unirlas para que todas queden en un solo numpy array. En nuestro caso, unimos las columnas que resultan del preprocesamiento de las categóricas y del de las numéricas.

In [58]:
from sklearn.pipeline import FeatureUnion

full_pipeline = ColumnTransformer(
    [
        ("num_pipeline", num_pipeline, num_cols),
        ("cat_pipeline", cat_pipeline, cat_cols)
    ]
)

X_transformed = full_pipeline.fit_transform(x_train, y_train)
print(X_transformed.shape)

2021/10/23 00:19:43 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID 'a8fa797b65564bb5ba0d0d70dc082a01', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current sklearn workflow
2021/10/23 00:19:43 WARNING mlflow.utils: Truncated the value of the key `steps`. Truncated value: `[('transformer_coord', ColumnTransformer(remainder='passthrough',
                  transformers=[('transformer_dist', TransformerDistancia(),
                                 ['pickup_longitude', 'pickup_latitude',
                               ...`
2021/10/23 00:19:43 WARNING mlflow.utils: Truncated the value of the key `transformer_coord`. Truncated value: `ColumnTransformer(remainder='passthrough',
                  transformers=[('transformer_dist', TransformerDistancia(),
                                 ['pickup_longitude', 'pickup_latitude',
                                  'dropoff_longitude',...`
2021/10/23 00:19:44 WAR

(1085891, 6)


    En este diagrama podemos ver que ejecutamos las transformaciones por separado para las variables numéricas y las categóricas y al final las unimos para tener los datos en un array que le pasaremos al modelo al momento de entrenar.

In [59]:
display(full_pipeline)

ColumnTransformer(transformers=[('num_pipeline',
                                 Pipeline(steps=[('transformer_coord',
                                                  ColumnTransformer(remainder='passthrough',
                                                                    transformers=[('transformer_dist',
                                                                                   TransformerDistancia(),
                                                                                   ['pickup_longitude',
                                                                                    'pickup_latitude',
                                                                                    'dropoff_longitude',
                                                                                    'dropoff_latitude'])])),
                                                 ('scaler', StandardScaler())]),
                                 ['passenger_count', 'pickup_longitude',
                                  'pickup_latitude', 'dropoff_longitude',
                                  'dropoff_latitude']),
                                ('cat_pipeline',
                                 Pipeline(steps=[('transformer_fechas',
                                                  ColumnTransformer(remainder='passthrough',
                                                                    transformers=[('transformer_fechas',
                                                                                   TransformerFechas(),
                                                                                   ['pickup_datetime'])])),
                                                 ('OrdinalEncoder',
                                                  OrdinalEncoder())]),
                                 ['vendor_id', 'pickup_borough',
                                  'pickup_datetime'])])

Guardar Pipeline a Disco

Además de tener una clara separación entre los métodos .fit() y .transform(), la ventaja de escribir todo nuestro proceso como un solo pipeline (full_pipeline) es que podemos guardar en disco el objeto que usamos para preprocesar. Esto es útil para tener todas las transformaciones definidas en un objeto al momento que queramos hacer reproducibles nuestras transformaciones para, por ejemplo, desplegar nuestro modelo.

Creamos un contexto usando open() y usamos la función dill.dump() para guardar nuestro flujo de preprocesamiento en disco.

In [60]:
import dill
dill.settings['recurse'] = True

with open("preprocesser.pkl", "wb") as f:
    dill.dump(full_pipeline, f) # guardar el flujo

print("Se ha guardado el pipeline !")

Se ha guardado el pipeline !


Para cargar nuestro flujo, usamos la función dill.load(). Además verificamos que el resultado de las transformaciones con el flujo que teníamos en el notebook y el que cargamos desde el disco es idéntico.

Checkpoint # 4

    Guardar pipeline de preprocesamiento en un archivo y volver a cargarlo con éxito.

In [61]:
with open("preprocesser.pkl", "rb") as f:
    loaded_pipeline = dill.load(f)
    
X_loaded = loaded_pipeline.transform(x_train)
print((X_loaded == X_transformed).all())

True


MLflow

Veremos cómo usar MLflow pra guardar métricas y parámetros de todos los modelos que corramos

In [62]:
with open("preprocesser.pkl", "rb") as f:
    preprocessor = dill.load(f)

X_train = preprocessor.transform(x_train)
X_test = preprocessor.transform(x_test)

# Entrenar Modelos

Baseline con DummyRegressor

Una opción que nos ofrece sklearn es entrenar modelos dummy. Estos son modelos que predicen lo mismo para todos los casos (en este caso el promedio) y nos sirven como una medida de cuál es el rendimiento mínimo para un problema. Si por alguna razón nuestro modelo tiene peor desempeño que el modelo dummy, tenemos que revisar nuestro modelo porque probablemente tenemos errores en la implementación.

En este caso vamos a usar el DummyRegressor que siempre predice la media de los datos de entrenamiento. La función evaluar_predicciones es una ayuda para poder calcular varias métricas de un modelo usando metricas incluídas de scikit-learn.

In [63]:
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, mean_squared_error, r2_score, mean_squared_log_error

def evaluar_predicciones(y_pred, y_true):
    mae = mean_absolute_error(y_pred=y_pred, y_true=y_true)
    mape = mean_absolute_percentage_error(y_pred=y_pred, y_true=y_true)
    rmse = mean_squared_error(y_pred=y_pred, y_true=y_true, squared=False)
    print(f"MAE: {mae:.2f}")
    print(f"MAPE: {mape}")
    print(f"RMSE: {rmse}")

DummyRegressor

In [64]:
from sklearn.dummy import DummyRegressor

dummy_model = DummyRegressor(strategy="mean")
dummy_model.fit(X_train, y_train)
y_train_dummy = dummy_model.predict(X_train)
y_test_dummy = dummy_model.predict(X_test)

print("TRAIN")
evaluar_predicciones(y_pred=y_train_dummy, y_true=y_train)

print("VALIDATION")
evaluar_predicciones(y_pred=y_test_dummy, y_true=y_test)

2021/10/23 00:20:09 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '4c4c7853f34c4f2a9aee55d02ff8897d', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current sklearn workflow


TRAIN
MAE: 468.61
MAPE: 0.95585624685749
RMSE: 683.1658889831872
VALIDATION
MAE: 468.42
MAPE: 0.9567650882594899
RMSE: 680.0030431781618


Baseline con regresión lineal

Otra buena medida es siempre empezar con un modelo lineal. Así tenemos una medida de cómo se desempeña un modelo sencillo.

Algo para notar es que sin importar el modelo, los modelos de sklearn siempre siguen el mismo proceso. 

1. Inicializar modelo con la clase de sklearn. 
2. Ejecutar función .fit(X_train, y_train. 
3. Ejecutar función .predict(X) para generar las predicciones.

Esta consistencia en el proceso simplifica el uso de diferentes modelos.

Checkpoint # 1

Entrenar un model de regresión linear usando LinearRergression de scikit-learn.Predecir valores del viaje para train (y_train_linear) y validación (y_val_linear)
 Las métricas del modelo deberían ser similares a las imagen de arriba.

In [65]:
from sklearn.linear_model import LinearRegression

linear_model = LinearRegression()
linear_model.fit(X_train, y_train)
y_train_linear = linear_model.predict(X_train)
y_test_linear = linear_model.predict(X_test)

print("TRAIN")
evaluar_predicciones(y_pred=y_train_linear, y_true=y_train)

print("VALIDATION")
evaluar_predicciones(y_pred=y_test_linear, y_true=y_test)

2021/10/23 00:20:14 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID 'ee5d1a9d2aa748198687c548607a9046', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current sklearn workflow


TRAIN
MAE: 296.09
MAPE: 0.5533021393418195
RMSE: 487.05043254440756
VALIDATION
MAE: 295.55
MAPE: 0.5540681395532686
RMSE: 529.0958979710379


¿Cómo comparar modelos más allá de un print? MLFlow 🙌

Imprimiendo los resultados nos podemos dar cuenta del modelo con mejores resultados. Sin embargo, comparar los resultados usando print es difícil cuando el número de modelos empieza a crecer. Además idealmente queremos que todos los modelos queden guardados para poder retomarlos después. Para todo esto podemos usar MLflow.
MLflow es una librería open-source que nos ayuda a manejar todo el ciclo de modelos de machine learning. En este caso vamos a ver cómo podemos usar MLflow para guardar resultados del desempeño de modelos para determinar cuál modelo es el mejor.
Vamos a correr los mismos modelos que acabamos de comparar pero esta vez vamos a usar MLflow para ver los resultados.

In [66]:
import mlflow
mlflow.sklearn.autolog()

In [67]:
pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


mlflow.sklearn.autolog() automáticamente nos ayuda a guardar varias métricas y parámetros del modelo. Sin embargo, no incluye valores para datos de validación. 
Para guardar métricas que no se incluyen en .autolog vamos a usar la función mlflow.log_metric dentro del context manager de run.

Vamos primero a usar el modelo dummy y vamos a poner run_name="dummy" para poder verlo con el nombre correcto en MLflow.

In [68]:
with mlflow.start_run(run_name="dummy") as run:
    dummy_model.fit(X_train, y_train)
    y_pred_val = dummy_model.predict(X_test)
    val_mae = mean_absolute_error(y_pred=y_pred_val, y_true=y_test)
    val_rmse = mean_squared_error(y_pred=y_pred_val, y_true=y_test, 
                                  squared=False)
    val_mape = mean_absolute_percentage_error(y_pred=y_pred_val, y_true=y_test)
    val_r2 = r2_score(y_pred=y_pred_val, y_true=y_test)

    mlflow.log_metric("val_mae", val_mae)
    mlflow.log_metric("val_rmse", val_rmse)
    mlflow.log_metric("val_mape", val_mape)
    mlflow.log_metric("val_r2", val_r2)

Una vez termine de correr esta celda debemos abrir la interfaz de MLflow para ver los resultados. Deberíamos encontrar una nueva carpeta llamada mlruns en la ubicación donde corrió este notebook. Una vez estemos en la carpeta donde está mlruns, debemos ir a un terminal a esa ubicación y ejecutamos el comando

In [69]:
# mlflow ui # para ejecutar este comando se debe ejecutar desde la carpeta principal

SyntaxError: invalid syntax (<ipython-input-69-8e7480d0936d>, line 1)

Ahí nos debería salir el mensaje de que está iniciando un servidor y dar la ruta para acceder. (normalmente la ruta es http://127.0.0.1:5000). Copiamos esa dirección en un navegador y ahí ya deberíamos ver la interfaz de MLflow.

![imagen mlflow](https://files.gitbook.com/v0/b/gitbook-28427.appspot.com/o/assets%2F-Mhiql7aQ7_XAS9SI7QS%2F-Mk98gvFzuPvmd1kfmM5%2F-Mk9LWf_4Z39Lr5ieALu%2Fimage.png?alt=media&token=42725dc4-0533-415d-b520-fc2327417908)

En la interfaz de MLflow estarán los modelos que corramos usando MLflow y podemos filtrar u ordenar por métricas o parámetros que consideramos importantes. 

En general, con la interfaz de MLflow tenemos una base de datos de modelos que podemos analizar fácilmente.

Además, si hacemos click un alguna de las runs (click en la columna Start Time) podemos ver que tenemos los datos más detallados y los artifacts de cada run. Los artifacts pueden ser todo tipo de archivos (por ejemplo imágenes y CSVs) que nos ayudan a guardar lo relevante de cada caso como el modelo o el ambiente de conda desde el que corrimos el modelo.

Ahora corramos nuevamente la regresión lineal para que quede registrada en MLflow. En este caso vamos a guardar nuestro preprocesador que quede asociado a cada run usando .log_artifact.

In [70]:
def log_metrics_mlflow(y_pred_val, y_test, name_modelo):
    print("log_metrics_mlflow >>", name_modelo)
    val_mae = mean_absolute_error(y_pred=y_pred_val, y_true=y_test)
    val_rmse = mean_squared_error(y_pred=y_pred_val, y_true=y_test, 
                                  squared=False)
    val_mape = mean_absolute_percentage_error(y_pred=y_pred_val, y_true=y_test)
    val_r2 = r2_score(y_pred=y_pred_val, y_true=y_test)

    mlflow.log_metric("val_mae", val_mae)
    mlflow.log_metric("val_rmse", val_rmse)
    mlflow.log_metric("val_mape", val_mape)
    mlflow.log_metric("val_r2", val_r2)
    mlflow.log_artifact("preprocesser.pkl")

In [71]:
with mlflow.start_run(run_name="linear_regression") as run:
    #TODO: Hacer fit a un modelo lineal
    linear_model.fit(X_train, y_train)
    y_pred_val = linear_model.predict(X_test)
    log_metrics_mlflow(y_pred_val, y_test, "linear_regression")
    mlflow.log_artifact("preprocesser.pkl")

log_metrics_mlflow >> linear_regression


Ahora que tenemos la regresión lineal y el modelo dummy en MLflow, vamos a probar con un modelo más complejo como el Random Forest. Por ahora vamos a dejar los parámetros por defecto que trae la clase RandomForestRegressor excepto n_jobs que lo vamos a poner con el parámetro n_jobs=2. Poner n_jobs=2 hace que el modelo corra en paralelo en 2 procesadores.
Entrenar el random forest puede tomar algunos minutos así que es un buen momento para resolver cualquier duda o tomar un descanso y estirar las piernas :)

In [ ]:
from sklearn.ensemble import RandomForestRegressor

with mlflow.start_run(run_name="random_forest") as run:
    #TODO: Hacer fit a un modelo random forest
    rf_model = RandomForestRegressor(n_jobs=2)
    rf_model.fit(X_train, y_train)
    y_pred_val = rf_model.predict(X_test)
    log_metrics_mlflow(y_pred_val, y_test, "random_forest")
    mlflow.log_artifact("preprocesser.pkl")

Con esas herramientas puedes buscar más modelos de regresión de sklearn o mejorar los hiperparámetros de los modelos que usamos para intentar mejorar las métricas que obtuvimos hasta el momento. Incluso librerías como XGBoost o LightGBM ofrecen la opción de crear modelos con la API de sklearn. A pesar de que cada modelo puede usar técnicas muy diferentes, el código va a ser casi idéntico gracias a la consistencia en la API de scikit-learn.